In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

/home/ez/.cache/pypoetry/virtualenvs/finetuning-lm-from-human-preferences-4SAAosyV-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [10]:
def apply_cot_few_shot_template(question):

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful reasoning assistant.\n"
                "ALWAYS follow the order of operations (BODMAS/PEMDAS):\n"
                "- First, evaluate Parentheses\n"
                "- Then Multiplication and Division (left to right)\n"
                "- Then Addition and Subtraction (left to right)\n"
                "Rewrite the expression at each step by substituting the evaluated part.\n"
                "Use multiple <think> ... </think> blocks.\n"
                "Conclude with <final_answer> ... </final_answer>."
            )
        },

        {
            "role": "user",
            "content": "What is (3 + 2) * 4?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: (3 + 2) * 4</think>\n"
                "<think>Evaluate (3 + 2): 5 → becomes 5 * 4</think>\n"
                "<think>Now multiply: 5 * 4 = 20</think>\n"
                "<final_answer>20</final_answer>"
            )
        },

        {
            "role": "user",
            "content": "What is 2 + 3 * 4?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: 2 + 3 * 4</think>\n"
                "<think>Multiplication before addition: 3 * 4 = 12 → becomes 2 + 12</think>\n"
                "<think>Now add: 2 + 12 = 14</think>\n"
                "<final_answer>14</final_answer>"
            )
        },

        {
            "role": "user",
            "content": "What is 12 / 4 + 5 - 1?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: 12 / 4 + 5 - 1</think>\n"
                "<think>Divide: 12 / 4 = 3 → becomes 3 + 5 - 1</think>\n"
                "<think>Add: 3 + 5 = 8 → becomes 8 - 1</think>\n"
                "<think>Subtract: 8 - 1 = 7</think>\n"
                "<final_answer>7</final_answer>"
            )
        },

        {
            "role": "user",
            "content": "What is ((2 + 3) * 2) + 1?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: ((2 + 3) * 2) + 1</think>\n"
                "<think>Inner parentheses: 2 + 3 = 5 → becomes (5 * 2) + 1</think>\n"
                "<think>Multiply: 5 * 2 = 10 → becomes 10 + 1</think>\n"
                "<think>Add: 10 + 1 = 11</think>\n"
                "<final_answer>11</final_answer>"
            )
        },

        {
            "role": "user",
            "content": "What is (6 + 2) * 3 / (2 + 2)?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: (6 + 2) * 3 / (2 + 2)</think>\n"
                "<think>Left parentheses: 6 + 2 = 8 → becomes 8 * 3 / (2 + 2)</think>\n"
                "<think>Right parentheses: 2 + 2 = 4 → becomes 8 * 3 / 4</think>\n"
                "<think>Multiply: 8 * 3 = 24 → becomes 24 / 4</think>\n"
                "<think>Divide: 24 / 4 = 6</think>\n"
                "<final_answer>6</final_answer>"
            )
        },

        {
            "role": "user",
            "content": "What is 10 / 2 * 3?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: 10 / 2 * 3</think>\n"
                "<think>Left to right: 10 / 2 = 5 → becomes 5 * 3</think>\n"
                "<think>Then: 5 * 3 = 15</think>\n"
                "<final_answer>15</final_answer>"
            )
        },

        {
            "role": "user",
            "content": "What is 10 - 2 + 4?"
        },
        {
            "role": "assistant",
            "content": (
                "<think>Original expression: 10 - 2 + 4</think>\n"
                "<think>Left to right: 10 - 2 = 8 → becomes 8 + 4</think>\n"
                "<think>Then: 8 + 4 = 12</think>\n"
                "<final_answer>12</final_answer>"
            )
        },

        # Your test query
        {
            "role": "user",
            "content": question
        }
    ]


    return messages

question = "What is 5 * (5 + 3)?"
text = tokenizer.apply_chat_template(apply_cot_few_shot_template(question), tokenize=False, add_generation_prompt=True)
inputs = tokenizer([text], return_tensors="pt")

output_ids = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=300,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

print(tokenizer.decode(output_ids[0]))

<|im_start|>system
You are a helpful reasoning assistant.
ALWAYS follow the order of operations (BODMAS/PEMDAS):
- First, evaluate Parentheses
- Then Multiplication and Division (left to right)
- Then Addition and Subtraction (left to right)
Rewrite the expression at each step by substituting the evaluated part.
Use multiple <think> ... </think> blocks.
Conclude with <final_answer> ... </final_answer>.<|im_end|>
<|im_start|>user
What is (3 + 2) * 4?<|im_end|>
<|im_start|>assistant
<think>Original expression: (3 + 2) * 4</think>
<think>Evaluate (3 + 2): 5 → becomes 5 * 4</think>
<think>Now multiply: 5 * 4 = 20</think>
<final_answer>20</final_answer><|im_end|>
<|im_start|>user
What is 2 + 3 * 4?<|im_end|>
<|im_start|>assistant
<think>Original expression: 2 + 3 * 4</think>
<think>Multiplication before addition: 3 * 4 = 12 → becomes 2 + 12</think>
<think>Now add: 2 + 12 = 14</think>
<final_answer>14</final_answer><|im_end|>
<|im_start|>user
What is 12 / 4 + 5 - 1?<|im_end|>
<|im_start|>ass

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What is 5 * (5 + 3)?"
    }
]


text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer([text], return_tensors="pt")

output_ids = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

print(tokenizer.decode(output_ids[0]))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is 5 * (5 + 3)?<|im_end|>
<|im_start|>assistant
The answer to the expression "5 * (5 + 3)" is 24.<|im_end|>
